Ce notebook permet de calculer des interval de normalsiation pour les scores finaux

In [1]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [2]:
twinewsScores = getTwinewsScores()
twinewsRankings = getTwinewsRankings()

twinews scores initialised.


In [3]:
scores = list(twinewsScores.find())
bp(scores)

[
  { _id: 5e7cd4158175111c4825e975, id: lda-aa8d5, metric: ndcg, score: 0.25 },
  { _id: 5e7cd41d8175111c4825e978, id: lda-aa8d5, metric: ndcg@10, score: 0.03 },
  ...,
  { _id: 5f19b4f1065f2836ed194791, id: Convnet-240ab, metric: p@100, score: 0.0 },
  { _id: 5f19b4f1065f2836ed194794, id: Convnet-240ab, metric: map, score: 0.01 }
]


In [4]:
# We filter scores:
nScores = []
errors = []
eliminated = []
for row in pb(scores):
    try:
        if twinewsRankings.getMeta(row['id'])['maxUsers'] is None:
            nScores.append(row)
        else:
            eliminated.append(row['id'])
    except Exception as e:
        errors.append(row['id'])
print("errors: " + b(errors))
print("eliminated: " + b(eliminated))
print("nScores: " + b(nScores))

  0% [                    ]
  9% [=                   ] (5m 19.702s left)
 19% [===                 ] (4m 44.382s left)
 29% [=====               ] (4m 8.966s left)
 39% [=======             ] (3m 33.292s left)
 49% [=========           ] (2m 57.582s left)
 59% [===========         ] (2m 21.889s left)
 69% [=============       ] (1m 46.419s left)
 79% [===============     ] (1m 10.9s left)
 89% [=================   ] (35.439s left)
 99% [=================== ] (0.022s left)
100% [====================] (total duration: 5m 54.11s, mean duration: 0.022s)
errors: [ ]
eliminated: [ tfidf-d0454, tfidf-d0454, ..., Convnet-240ab, Convnet-240ab ]
nScores: [
  { _id: 5e7cd4158175111c4825e975, id: lda-aa8d5, metric: ndcg, score: 0.25 },
  { _id: 5e7cd41d8175111c4825e978, id: lda-aa8d5, metric: ndcg@10, score: 0.03 },
  ...,
  { _id: 5ed76698065f281b7bb6a9cf, id: BM25_yf-72299, metric: p@100, score: 0.04 },
  { _id: 5ed766a9065f281b7bb6a9d2, id: BM25_yf-72299, metric: map, score: 0.28 }
]


In [5]:
scores = nScores

In [6]:
metrics = set()
for row in scores:
    metrics.add(row['metric'])
bp(metrics)

{ bm25-ser@100, div@100, ..., topic-snov@100, wtfidf-ser@100 }


In [7]:
allowedMetrics = \
{
    ### Ranking accuracy:
    # 'ndcg', 'ndcg@10', 'ndcg@100', 'map', 'mrr', 'p@10', 'p@100',
    ### Diversity:
    # 'div@100', 'topic-div@100', 'jacc-div@100', 'swjacc-div@100', 'style-div@100',
    ### Novelty:
    'nov@100', 'topic-nov@100', 'jacc-nov@100', 'swjacc-nov@100', 'style-nov@100',
    # ### Strict novelty:
    # 'snov@100', 'topic-snov@100', 'jacc-snov@100', 'swjacc-snov@100', 'style-snov@100',
    ### Serendipity:
    # 'tfidf-ser@100', 'wtfidf-ser@100', 'bm25-ser@100', 'jacc-ser@100', 'style-ser@100',
}

In [8]:
data = []
for metric in metrics:
    if metric in allowedMetrics:
        mini = None
        miniId = None
        maxi = None
        maxiId = None
        for row in scores:
            score = row['score']
            currentMetric = row['metric']
            if currentMetric == metric:
                if mini is None or score < mini: # and ((not "ser@" in metric) or score > 0.0001)
                    mini = score
                    miniId = row['id']
                if maxi is None or score > maxi:
                    maxi = score
                    maxiId = row['id']
        floatTruncation = 2
        nmini = truncateFloat(mini, 1)
        if nmini < 0.0:
            nmini = 0.0
        nmaxi = truncateFloat(maxi, 1) + 0.2
        if nmaxi > 1.0:
            nmaxi = 1.0
        data.append\
        (
            {
                "metric": metric,
                "min": truncateFloat(mini, floatTruncation), "max": truncateFloat(maxi, floatTruncation),
                "nmin": nmini, "nmax": nmaxi,
                "minId": miniId, "maxId": maxiId,
            }
        )
displayDicts(data, start=['metric', 'minId', 'min', 'nmin', 'maxId', 'max', 'nmax'], blue={'min', 'max'}, green=['maxId', 'minId'], red=['nmin', 'nmax'])

,metric,minId,min,nmin,maxId,max,nmax
0,topic-nov@100,nmf-6078e,0.75,0.7,worst-559a0,0.89,1
1,style-nov@100,dbert-ft-3a57f,0.49,0.4,bm25-c4813,0.61,0.8
2,nov@100,tfidf-7febb,0.94,0.9,worst-559a0,0.97,1
3,jacc-nov@100,tfidf-4c997,0.87,0.8,bm25-5b0c5,0.9,1
4,swjacc-nov@100,tfidf-7e79d,0.95,0.9,bm25-c4813,0.97,1
